## 1. Objective
<!-- trying to frame problem statement, define scope -->
Design and demonstrate an **agentic workflow** capable of answering user queries related to Singapore public transport. Primary focus will be on **real-time bus arrival information**.
Agent will dynamically decide when, how to fetch data from APIs and generate responses via LLM.

## 2. Agent responsibilities
- interpret user queries (in natural language)
- identify user intent, relevant entities
- dynamically invoke LTA APIs
- ?? incorporate constraints depending on context ?? (eg - time of the day, weather conditions?, holidays?)
- generate llm response
- use langgraph to showcase clear, modular workflow
